<a href="https://colab.research.google.com/github/bhanuteja2001/SegPC-2021/blob/main/Preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/SegPC-2021

/content/drive/MyDrive/SegPC-2021


In [ ]:
!unzip "/content/drive/MyDrive/SegPC-2021/SegPC_2021_Train_data.zip"

In [7]:
import os
import os
import random
import re
from PIL import Image

DATA_PATH = '/content/drive/MyDrive/SegPC-2021/train/'
FRAME_PATH = "/content/drive/MyDrive/SegPC-2021/train/x/"
MASK_PATH = "/content/drive/MyDrive/SegPC-2021/train/y/"

In [ ]:
os.listdir("/content/drive/MyDrive")

['covid_image_data', 'Colab Notebooks', 'SegPC-2021']

In [ ]:
len(os.listdir("/content/drive/MyDrive/SegPC-2021/train/x"))

297

In [8]:
all_frames = os.listdir(FRAME_PATH)
all_masks = os.listdir(MASK_PATH)

In [14]:
def fin(masks,frame):
  em = []
  for f in masks:
    f_1 = f[0:3]+".bmp"
    if f_1 in frame:
      em.append(f)
  return em

In [51]:
# Create folders to hold images and masks

folders = ['train_frames', 'train_masks', 'val_frames', 'val_masks', 'test_frames', 'test_masks']


for folder in folders:
  os.makedirs(DATA_PATH + folder)
  


# Get all frames and masks, sort them, shuffle them to generate data sets.

all_frames = os.listdir(FRAME_PATH)
all_masks = os.listdir(MASK_PATH)


all_frames.sort(key=lambda var:[int(x) if x.isdigit() else x 
                                for x in re.findall(r'[0-9]+', var)])
all_masks.sort(key=lambda var:[int(x) if x.isdigit() else x 
                               for x in re.findall(r'[0-9]+', var)])


random.seed(230)
random.shuffle(all_frames)


# Generate train, val, and test sets for frames

train_split = int(0.7*len(all_frames))
val_split = int(0.9 * len(all_frames))

train_frames = all_frames[:train_split]
val_frames = all_frames[train_split:val_split]
test_frames = all_frames[val_split:]


# Generate corresponding mask lists for masks

train_masks = fin(all_masks,train_frames)
val_masks = fin(all_masks,val_frames)
test_masks = fin(all_masks,test_frames)

In [52]:
frame_folders = [(train_frames, 'train_frames'), (val_frames, 'val_frames'), 
                 (test_frames, 'test_frames')]

mask_folders = [(train_masks, 'train_masks'), (val_masks, 'val_masks'), 
                (test_masks, 'test_masks')]

In [31]:
name[1][0:4]

'test'

In [53]:
#Add train, val, test frames and masks to relevant folders


def add_frames(dir_name, image):
  
  img = Image.open(FRAME_PATH+image)
  img.save(DATA_PATH+'/{}'.format(dir_name)+'/'+image)
  
  
  
def add_masks(dir_name, image):
  
  img = Image.open(MASK_PATH+image)
  img.save(DATA_PATH+'/{}'.format(dir_name)+'/'+image)



# Add frames

for folder in frame_folders:
  
  array = folder[0]
  name = [folder[1]] * len(array)

  list(map(add_frames, name, array))
         
    
# Add masks

for folder in mask_folders:
  
  array = folder[0]
  name = [folder[1]] * len(array)
  
  list(map(add_masks, name, array))

In [54]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        channel_shift_range=150.0,
        vertical_flip=True,
        horizontal_flip=True)
        
val_datagen = ImageDataGenerator(rescale=1./255)

In [55]:
train_image_generator = train_datagen.flow_from_directory(
    directory='/content/drive/MyDrive/SegPC-2021/train/train_frames/',
    batch_size = 16)

train_mask_generator = train_datagen.flow_from_directory(
    directory='/content/drive/MyDrive/SegPC-2021/train/train_masks/',
    batch_size = 16)

val_image_generator = val_datagen.flow_from_directory(
    directory='/content/drive/MyDrive/SegPC-2021/train/val_frames/',
    batch_size = 16)


val_mask_generator = val_datagen.flow_from_directory(
    directory='/content/drive/MyDrive/SegPC-2021/train/val_masks/',
    batch_size = 16)



train_generator = zip(train_image_generator, train_mask_generator)
val_generator = zip(val_image_generator, val_mask_generator)

Found 208 images belonging to 1 classes.
Found 118 images belonging to 1 classes.
Found 60 images belonging to 1 classes.
Found 68 images belonging to 1 classes.


In [ ]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import CSVLogger
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam

import model

NO_OF_TRAINING_IMAGES = len(os.listdir('/your_data/train_frames/train/'))
NO_OF_VAL_IMAGES = len(os.listdir('/your_data/val_frames/val/'))

NO_OF_EPOCHS = 'ANYTHING FROM 30-100 FOR SMALL-MEDIUM SIZED DATASETS IS OKAY'

BATCH_SIZE = 'BATCH SIZE PREVIOUSLY INITIALISED'

weights_path = 'path/where/resulting_weights_will_be_saved'

m = model.FCN_Vgg16_32s()
opt = Adam(lr=1E-5, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

m.compile(loss='The loss to optimise [eg: dice_loss],
              optimizer=opt,
              metrics='YOUR_METRIC [could be 'accuracy' or mIOU, dice_coeff etc]')

checkpoint = ModelCheckpoint(weights_path, monitor='METRIC_TO_MONITOR', 
                             verbose=1, save_best_only=True, mode='max')

csv_logger = CSVLogger('./log.out', append=True, separator=';')

earlystopping = EarlyStopping(monitor = 'METRIC_TO_MONITOR', verbose = 1,
                              min_delta = 0.01, patience = 3, mode = 'max')

callbacks_list = [checkpoint, csv_logger, earlystopping]

results = m.fit_generator(train_gen, epochs=NO_OF_EPOCHS, 
                          steps_per_epoch = (NO_OF_TRAINING_IMAGES//BATCH_SIZE),
                          validation_data=val_gen, 
                          validation_steps=(NO_OF_VAL_IMAGES//BATCH_SIZE), 
                          callbacks=callbacks_list)
m.save('Model.h5')